In [1]:
import pandas as pd
import numpy as np
import json
import folium
from helper_funcs import *

In [2]:
RF = get_redfin_csv()
RF = RF[['zip','State','PRICE']]

----pulling Redfin data from Azure storage----
----Done----


In [3]:
RF = RF.groupby(['State','zip']).agg('mean')
RF.PRICE = RF.PRICE.apply(round)
RF = RF.reset_index()

In [4]:
RF.zip = RF.zip.astype('string')
RF

,State,zip,PRICE
0,DC,20001,812526
1,DC,20002,787511
2,DC,20003,947111
3,DC,20004,728216
4,DC,20005,846988
...,...,...,...
2338,WV,26836,288491
2339,WV,26845,287475
2340,WV,26847,235829
2341,WV,26851,191333


In [5]:
with open('C:/Users/dn-83/Documents/bootcamp/capstone_proj/_data/dc_district_of_columbia_zip_codes_geo.min.json') as f1:
    the_geodata = json.load(f1)
f1.close()

In [6]:
m = folium.Map(location=[38.5976, -80.4549],    
               tiles="cartodbpositron",
               zoom_start=10)

style1 = {'fillColor': '#228B22', 'color': '#808080', 'weight':'1'}
style2 = {'fillColor': '#3366CC', 'color': '#FFFFFF', 'weight':'1'}

# folium.GeoJson(dc_geodata, name="dc_geojson",
#               style_function=lambda x: style1,
#               ).add_to(m)

folium.Choropleth(geo_data=the_geodata,
                  name="WV",
                  data=RF[RF.State=='WV'],
                  columns=["zip", "PRICE"],
                  key_on="feature.properties.ZCTA5CE10",
                  fill_color="YlGnBu",
                  fill_opacity=0.6,
                  line_opacity=0.5,
                  legend_name="Average PRICE",
                 ).add_to(m)

# points_list = hosp[['latitude','longitude']].values.tolist()

# for point in range(0, len(points_list)):
#     folium.Marker(points_list[point], popup=hosp['Facility_Name'][point]).add_to(m)



style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
hover = folium.features.GeoJson(the_geodata,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['ZCTA5CE10'],
        aliases=[''],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 14px; padding: 1px;") 
    )
)
m.add_child(hover)
m.keep_in_front(hover)
folium.LayerControl().add_to(m)

m

In [30]:
m.save("WV_map.html")